# Gabor filters
* http://www.cs.rug.nl/~imaging/simplecell.html
* https://scikit-image.org/docs/0.11.x/auto_examples/plot_gabor.html
* https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_gabor.html
* http://vision.psych.umn.edu/users/kersten/kersten-lab/courses/Psy5036W2017/Lectures/17_PythonForVision/Demos/html/2b.Gabor.html
* https://medium.com/@anuj_shah/through-the-eyes-of-gabor-filter-17d1fdb3ac97
* https://dsp.stackexchange.com/questions/14714/understanding-the-gabor-filter-function
* [Extracting Gabors from DoG filtered images with k-means](https://scikit-image.org/docs/0.11.x/auto_examples/plot_gabors_from_astronaut.html)
* [Notes including temporal structure](http://www.inf.ed.ac.uk/teaching/courses/nc/NClab5.pdf)

# Difference of Gaussian filters
* https://github.com/heitorrapela/xdog/blob/master/main.py
* https://webvision.med.utah.edu/book/part-ii-anatomy-and-physiology-of-the-retina/ganglion-cell-physiology/

# Custom kernels
* https://keras.io/layers/writing-your-own-keras-layers/
* https://keras.io/layers/core/ (See Lambda)
* https://stackoverflow.com/questions/51265578/keras-conv2d-custom-kernel-initialization
* https://keunwoochoi.wordpress.com/2016/11/18/for-beginners-writing-a-custom-keras-layer/

## Replacing layers
* https://stackoverflow.com/questions/49492255/how-to-replace-or-insert-intermediate-layer-in-keras-model

# Gabor networks
* https://ieeexplore.ieee.org/document/8009202
* https://www.mdpi.com/2079-9292/8/1/105
* https://arxiv.org/pdf/1904.13204.pdf

# Initialise a TensorFlow session
* https://stackoverflow.com/questions/34097281/how-can-i-convert-a-tensor-into-a-numpy-array-in-tensorflow

# Fit generator
* https://www.pyimagesearch.com/2018/12/24/how-to-use-keras-fit-and-fit_generator-a-hands-on-tutorial/

In [ ]:
%matplotlib inline
import os
import keras
from keras import backend as K
from keras.preprocessing import image
import tensorflow as tf
from matplotlib import pyplot as plt
import cv2
import numpy as np


def calc_bandwidth(lambd, sigma):
    r = np.pi*sigma/lambd
    c = np.sqrt(np.log(2)/2)
    return np.log2((r + c)/(r - c))

def calc_sigma(lambd, bandwidth):
    p = 2**bandwidth
    c = np.sqrt(np.log(2)/2)
    return lambd * c / np.pi  * (p + 1) / (p - 1)

def calc_lambda(sigma, bandwidth):
    p = 2**bandwidth
    c = np.sqrt(np.log(2)/2)
    return sigma * np.pi / c  * (p - 1) / (p + 1)

Here we implement a type of Gabor filter which satisfies the neurophysiological constraints for simple cells:
$\psi (x; \omega, \theta, K) = \left[\frac{\omega^2}{ 4 \pi K^2} \exp  \{-(\omega^2/8K^2)[4(x\cdot(cos\theta, sin\theta))^2 + (x \cdot ( -sin \theta, cos \theta))^2]\} \right] \times \left[ \exp \{ iwx \cdot (cos\theta, sin\theta) \} exp(K^2/2) \right]$

In [ ]:
# Simple cell model: http://vision.psych.umn.edu/users/kersten/kersten-lab/courses/Psy5036W2017/Lectures/17_PythonForVision/Demos/html/2b.Gabor.html
def genGabor(sz, omega, theta, func=np.cos, K=np.pi):
    radius = (int(sz[0]/2.0), int(sz[1]/2.0))
    [x, y] = np.meshgrid(range(-radius[0], radius[0]+1), range(-radius[1], radius[1]+1))

    x1 = x * np.cos(theta) + y * np.sin(theta)
    y1 = -x * np.sin(theta) + y * np.cos(theta)
    
    gauss = omega**2 / (4*np.pi * K**2) * np.exp(- omega**2 / (8*K**2) * ( 4 * x1**2 + y1**2))
#     myimshow(gauss)
    sinusoid = func(omega * x1) * np.exp(K**2 / 2)
#     myimshow(sinusoid)
    gabor = gauss * sinusoid
    return gabor
        
g = genGabor((256,256), 0.3, np.pi/4, func=np.cos) 
# change func to "cos", "sin" can generate sin gabor or cos gabor, here we pass a function name as a parameter
myimshow(g)
np.mean(g)

In [ ]:
# Set parameters
project_root = os.path.realpath(os.pardir)
fig_dir = os.path.join(project_root, "results", "figs")
if not os.path.isdir(fig_dir):
    os.makedirs(fig_dir)

In [ ]:
image_path = "/work/data/Lenna.png"
# image_path = "/work/data/example_aeroplane_s_000021.png"
# image_path = "/work/data/4.2.03.tiff"  # Mandrill
# image_path = "/work/data/4.2.07.tiff"  # Peppers
# img = image.load_img(image_path)
# img = plt.imread(image_path)
img = cv2.imread(image_path)
# img = np.asarray(img, dtype=np.uint8)

print(np.amin(img), np.amax(img))
print(img.shape)

img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = np.copy(img.astype('float32')) / 255
# img = img.astype('float32')
print(type(img))
img = K.expand_dims(img, 0)
img = K.expand_dims(img, -1)
print(img.shape)
print(np.amin(img), np.amax(img))

lambdas = [3, 4, 5, 6, 7, 8]  # 3 <= lambd <= W/2
# sigmas = [1, 2, 3] # 4]  
# bandwidths = np.linspace(0.4, 2.6, num=3)  # ~1.5 <= bw <= ~3
bandwidths = np.linspace(1, 1.8, num=3)

lambdas = [5]
bandwidths = [1.4]

n_thetas = 8
n_psis = 4  # 1, 2, 4
n_gammas = 2

thetas = np.linspace(0, np.pi, n_thetas, endpoint=False)
psis = np.linspace(0, 2*np.pi, n_psis, endpoint=False)
gammas = np.linspace(1, 0, n_gammas, endpoint=False)

# Fix sigma and bw
sigmas = [0.2, 0.4, 0.6, 0.8, 1, 2, 3, 4, 5]
bandwidths = np.linspace(1, 1.8, num=3)
# thetas = np.linspace(0, np.pi, 4, endpoint=False)
gammas = [0.5]
# psis = np.linspace(0, 2*np.pi, 2, endpoint=False)


convolve = True
size = (31, 31)
# gabor = {'sigmas': [sigma],
#          'lambdas': [lambd],
#          'thetas': thetas,        
#          'psis': psis,
#          'gammas': gammas,
#          'ksize': (31, 31),
#         }

ncols = len(thetas)
nrows = int(np.ceil(len(gammas)*len(psis)*len(sigmas)*len(bandwidths)))

fontsize = 20
space = 0.15
width = 24
print(f"Total Gabor filters: {ncols*nrows}")

img_scale = 6

if convolve:
    nrows *= 2

i = 0
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex='row', sharey='row', figsize=(width, width*nrows/ncols))
for sg, sigma in enumerate(sigmas):
    for bw in bandwidths:
#     for lm, lambd in enumerate(lambdas):
#         sigma = calc_sigma(lambd, bw)
        lambd = calc_lambda(sigma, bw)
        print(f"{float(sigma):<3}, {bw}, {calc_lambda(sigma, bw):<05.3}")
#         fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex='row', sharey='row', figsize=(width, width*nrows/ncols))
        
        for gm, gamma in enumerate(gammas):
            for ps, psi in enumerate(psis):
                for th, theta in enumerate(thetas):

                    params = {'ksize': size, 'sigma': sigma, 'theta': theta, 'lambd': lambd, 'gamma': gamma, 'psi': psi}
                    gf = cv2.getGaborKernel(**params, ktype=cv2.CV_32F)

                    row, col = (i//ncols), i%ncols
                    if convolve:
                        row *= 2
                    axes[row, col].imshow(gf, cmap='gray', vmin=-1, vmax=1)
                    axes[row, col].set_xticks([])
                    axes[row, col].set_yticks([])
                    # print(np.amin(gf), np.amax(gf))
#                     simplify(th*np.pi/n_thetas)
                    if i//ncols == 0:
#                         axes[row, col].set_title(r"$\theta = {:.3}\pi$".format(theta/np.pi))

                        if th == 0:
                            axes[row, col].set_title(r"$\theta = 0$", fontsize=fontsize)
                        else:
                            axes[row, col].set_title(r"$\theta = \frac{{{}}}{{{}}}\pi$".format(th, n_thetas), fontsize=fontsize)
                    if i%ncols == 0:
#                         axes[row, col].set_ylabel(r"$\psi = {:.3}\pi, \gamma = {}$".format(psi/np.pi, gamma))  #lambd, sigma))
                        if ps == 0:
                            axes[row, col].set_ylabel(r"$\psi = 0, \gamma = {}$".format(gamma), fontsize=fontsize)
                        else:
                            axes[row, col].set_ylabel(r"$\psi = \frac{{{}}}{{{}}}\pi, \gamma = {}$".format(ps, n_psis, gamma), fontsize=fontsize)

#                     print(f"C: {channel}; sigma_c: {float(sigma):.1f}; r_sigma: {sig_ratio:.3f}; [{np.amin(dog):.5f}, {np.amax(dog):.5f}], Sum: {np.sum(dog):.5}", end='')
                    if convolve:
                        gf = K.expand_dims(gf, -1)
                        gf = K.expand_dims(gf, -1)
                        # https://stackoverflow.com/questions/34619177/what-does-tf-nn-conv2d-do-in-tensorflow
                        # K.conv2d(image.img_to_array(img), gf)
                        fimg = K.conv2d(img, gf, padding='same')
                        fimg = tf.Session().run(fimg[0,:,:,0])
#                         fimg = np.sign(fimg) * np.log(np.abs(fimg))  # Logarithmically compress convolved values
#                         fimg /= np.amax(np.abs(fimg))  # Scale to [-1, 1]
                        axes[row+1, col].imshow(fimg, cmap='gray') #, vmin=-img_scale, vmax=img_scale)
                        # axes[row+1, col].imshow(fimg[0,:,:,0].eval(), cmap='gray')
                        axes[row+1, col].set_xticks([])
                        axes[row+1, col].set_yticks([])
                        
                        if i%ncols == 0:
                            axes[row+1, col].set_ylabel(r"$\lambda = {:.2f}, \sigma = {:.0f}, b = {:.1f}$".format(lambd, float(sigma), bw), fontsize=fontsize)
#                         print(np.amin(fimg), np.amax(fimg))
                    i += 1
plt.tight_layout()
plt.subplots_adjust(wspace=space, hspace=space)
# plt.savefig(os.path.join(fig_dir, f"gabor_kernels.pdf"), bbox_inches="tight")  # , additional_artists=[lgd])

In [ ]:
image_path = "/work/data/Lenna.png"
image_path = "/work/data/example_aeroplane_s_000021.png"
# image_path = "/work/data/4.2.03.tiff"  # Mandrill
# image_path = "/work/data/4.2.07.tiff"  # Peppers
# img = image.load_img(image_path)
# img = plt.imread(image_path)
img = cv2.imread(image_path)
# img = np.asarray(img, dtype=np.uint8)

print(np.amin(img), np.amax(img))
print(img.shape)

img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = np.copy(img.astype('float32')) / 255
# img = img.astype('float32')
print(type(img))
img = K.expand_dims(img, 0)
img = K.expand_dims(img, -1)
print(img.shape)
print(np.amin(img), np.amax(img))

n_channels = 2  # On and Off centre
channels = [1, -1]
sigmas_c = [0.25, 0.5, 1, 2, 3, 4, 5]
# r_sigmas = np.logspace(np.log10(1.05), np.log10(3), 3)  # sigma_c
r_sigmas = [1.1, 1.2, 1.5, 2, 2.5, 3]
# r_gammas = np.logspace(-0.05, 0.05, 5)
r_gammas = [1]

convolve = True
ksize = 31  # (31, 31)

ncols = int(len(channels)*len(r_sigmas))
nrows = int(np.ceil(len(sigmas_c)*len(r_gammas)))

fontsize = 20
space = 0.15
width = 24
print(f"Total DoG filters: {ncols*nrows}")

img_scale = 6

if convolve:
    nrows *= 2

i = 0
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex='row', sharey='row', figsize=(width, width*nrows/ncols))

for rg, gam_ratio in enumerate(r_gammas):        
    for sg, sigma in enumerate(sigmas_c):
        for rs, sig_ratio in enumerate(r_sigmas):
            for ch, channel in enumerate(channels):
#             print(f"{float(sigma):<3}, {bw}, {calc_lambda(sigma, bw):<05.3}")

                g_c = cv2.getGaussianKernel(ksize, sigma, ktype=cv2.CV_32F)
                g_s = gam_ratio * cv2.getGaussianKernel(ksize, sig_ratio*sigma, ktype=cv2.CV_32F)
                dog = g_c - g_s
#                 dog /= np.sum(dog)  # Normalise
                dog *= channel
                print(f"C: {channel}; sigma_c: {float(sigma):.1f}; r_sigma: {sig_ratio:.3f}; [{np.amin(dog):.5f}, {np.amax(dog):.5f}], Sum: {np.sum(dog):.5}", end='')

                row, col = (i//ncols), i%ncols
                if convolve:
                    row *= 2
                axes[row, col].imshow(dog.T, cmap='gray', vmin=-1, vmax=1)
                axes[row, col].set_xticks([])
                axes[row, col].set_yticks([])

                if i//ncols == 0:
                    axes[row, col].set_title(r"$C_{{{}}}, r_\sigma = {:.3}$".format(channel, float(sig_ratio)), fontsize=fontsize)

                if i%ncols == 0:
                    axes[row, col].set_ylabel(r"$\sigma_c = {:.3}, r_\gamma = {:.3}$".format(float(sigma), float(gam_ratio)), fontsize=fontsize, rotation=0)

                if convolve:
                    dog = K.expand_dims(dog, -1)
                    dog = K.expand_dims(dog, -1)
                    # https://stackoverflow.com/questions/34619177/what-does-tf-nn-conv2d-do-in-tensorflow
                    # K.conv2d(image.img_to_array(img), gf)
                    fimg = K.conv2d(img, dog, padding='same')
                    fimg = tf.Session().run(fimg[0,:,:,0])
#                     fimg /= np.amax(np.abs(fimg))  # Scale to [-1, 1]
                    axes[row+1, col].imshow(fimg, cmap='gray') #, vmin=-img_scale, vmax=img_scale)
                    # axes[row+1, col].imshow(fimg[0,:,:,0].eval(), cmap='gray')
                    axes[row+1, col].set_xticks([])
                    axes[row+1, col].set_yticks([])
                    print(f" --> [{np.amin(fimg):.5f}, {np.amax(fimg):.5f}], Sum: {np.sum(fimg):.5}")
#                     if i%ncols == 0:
#                         axes[row+1, col].set_ylabel(r"$\lambda = {:.2f}, \sigma = {:.0f}, b = {:.1f}$".format(lambd, float(sigma), bw), fontsize=fontsize)
                else:
                    print()
                i += 1
plt.tight_layout()
plt.subplots_adjust(wspace=space, hspace=space)
plt.savefig(os.path.join(fig_dir, f"dog_kernels.pdf"), bbox_inches="tight")  # , additional_artists=[lgd])

In [ ]:
image_path = "/work/data/Lenna.png"
# img = image.load_img(image_path)
img = plt.imread(image_path)
print(img.shape)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
print(type(img))
img = K.expand_dims(img, 0)
img = K.expand_dims(img, -1)
print(img.shape)

lambdas = [3, 4, 5, 6, 7, 8]  # 3 <= lambd <= W/2
sigmas = [1, 2, 3] # 4]  
# bandwidths = np.linspace(0.4, 2.6, num=3)  # ~1.5 <= bw <= ~3
bandwidths = np.linspace(1, 1.8, num=3)

n_thetas = 8
n_psis = 4  # 1, 2, 4
n_gammas = 2

thetas = np.linspace(0, np.pi, n_thetas, endpoint=False)
psis = np.linspace(0, 2*np.pi, n_psis, endpoint=False)
gammas = np.linspace(1, 0, n_gammas, endpoint=False)

size = (31, 31)
# gabor = {'sigmas': [sigma],
#          'lambdas': [lambd],
#          'thetas': thetas,        
#          'psis': psis,
#          'gammas': gammas,
#          'ksize': (31, 31),
#         }

ncols = len(thetas)
nrows = int(np.ceil(2*len(gammas)*len(sigmas)*len(thetas)*len(lambdas)*len(psis)/ncols))
width = 24
print(f"Total Gabor filters: {ncols*nrows}")

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex='row', sharey='row', figsize=(width, width*nrows/ncols))
i = 0
#     for sg, sigma in enumerate(sigmas):
for bw in bandwidths:
    for lm, lambd in enumerate(lambdas):
        sigma = calc_sigma(lambd, bw)
        for gm, gamma in enumerate(gammas):
            for ps, psi in enumerate(psis):
                for th, theta in enumerate(thetas):

                    params = {'ksize': size, 'sigma': sigma, 'theta': theta, 'lambd': lambd, 'gamma': gamma, 'psi': psi}
                    gf = cv2.getGaborKernel(**params, ktype=cv2.CV_32F)

                    row, col = 2*(i//ncols), i%ncols
                    axes[row, col].imshow(gf, cmap='gray', vmin=-1, vmax=1)
                    axes[row, col].set_xticks([])
                    axes[row, col].set_yticks([])

                    if i//ncols == 0:
                        axes[row, col].set_title(r"$\theta = {:.3}\pi$".format(theta/np.pi))
                    if i%ncols == 0:
                        axes[row, col].set_ylabel(r"$\psi = {:.3}\pi, \lambda = {}, \sigma = {:4.3}$".format(psi/np.pi, lambd, sigma))

                    gf = K.expand_dims(gf, -1)
                    gf = K.expand_dims(gf, -1)
                    # https://stackoverflow.com/questions/34619177/what-does-tf-nn-conv2d-do-in-tensorflow
                    # K.conv2d(image.img_to_array(img), gf)
                    fimg = K.conv2d(img, gf, padding='same')
                    axes[row+1, col].imshow(tf.Session().run(fimg[0,:,:,0]), cmap='gray')
                    # axes[row+1, col].imshow(fimg[0,:,:,0].eval(), cmap='gray')
                    axes[row+1, col].set_xticks([])
                    axes[row+1, col].set_yticks([])
                    i += 1

In [ ]:
image_path = "/work/data/Lenna.png"
# img = image.load_img(image_path)
img = plt.imread(image_path)
print(img.shape)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
print(type(img))
img = K.expand_dims(img, 0)
img = K.expand_dims(img, -1)
print(img.shape)

# ksize is the size of the Gabor kernel. If ksize = (a, b), we then have a Gabor kernel of size a x b pixels. As with many other convolution kernels, ksize is preferably odd and the kernel is a square (just for the sake of uniformity).
# sigma is the standard deviation of the Gaussian function used in the Gabor filter.
# theta is the orientation of the normal to the parallel stripes of the Gabor function.
# lambda is the wavelength of the sinusoidal factor in the above equation.
# gamma is the spatial aspect ratio.
# psi is the phase offset.
# ktype indicates the type and range of values that each pixel in the Gabor kernel can hold.

sizes = [(31, 31)] #[(25, 25)]  # [(5, 5), (15, 15), (25, 25)]
# sigmas = [2, 4]
thetas = np.linspace(0, 2*np.pi, 8, endpoint=False)  # [0, np.pi/4, np.pi/2, np.pi*3/4]
# lambdas = [8] #, 16]
lambdas = [3, 4, 5, 6, 7, 8]  # 3 <= lambd <= W/2
sigmas = [1, 2, 3] # 4]  
# bandwidths = np.linspace(0.4, 2.6, num=3)  # ~1.5 <= bw <= ~3
bandwidths = np.linspace(1, 1.8, num=3)
psis = np.linspace(0, 2*np.pi, 4, endpoint=False)
# psis = [0, np.pi/2, np.pi, 3*np.pi/2]
gamma = 0.5

ncols = len(thetas)
nrows = int(np.ceil(2*len(sizes)*len(sigmas)*len(thetas)*len(lambdas)*len(psis)/ncols))
width = 24
print(f"Total Gabor filters: {ncols*nrows}")

# sess = tf.InteractiveSession()
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex='row', sharey='row', figsize=(width, width*nrows/ncols))
i = 0
for sz, size in enumerate(sizes):
#     for sg, sigma in enumerate(sigmas):
    for bw in bandwidths:
        for lm, lambd in enumerate(lambdas):
            sigma = calc_sigma(lambd, bw)
            for ps, psi in enumerate(psis):
                for th, theta in enumerate(thetas):

                    params = {'ksize': size, 'sigma': sigma, 'theta': theta, 'lambd': lambd, 'gamma': gamma, 'psi': psi}
                    gf = cv2.getGaborKernel(**params, ktype=cv2.CV_32F)

                    row, col = 2*(i//ncols), i%ncols
                    axes[row, col].imshow(gf, cmap='gray')
                    axes[row, col].set_xticks([])
                    axes[row, col].set_yticks([])

                    if i//ncols == 0:
                        axes[row, col].set_title(r"$\theta = {:.3}\pi$".format(theta/np.pi))
                    if i%ncols == 0:
                        axes[row, col].set_ylabel(r"$\psi = {:.3}\pi, \lambda = {}, \sigma = {:4}$".format(psi/np.pi, lambd, sigma))

                    gf = K.expand_dims(gf, -1)
                    gf = K.expand_dims(gf, -1)
                    # https://stackoverflow.com/questions/34619177/what-does-tf-nn-conv2d-do-in-tensorflow
                    # K.conv2d(image.img_to_array(img), gf)
                    fimg = K.conv2d(img, gf, padding='same')
                    axes[row+1, col].imshow(tf.Session().run(fimg[0,:,:,0]), cmap='gray')
                    # axes[row+1, col].imshow(fimg[0,:,:,0].eval(), cmap='gray')
                    axes[row+1, col].set_xticks([])
                    axes[row+1, col].set_yticks([])
                    i += 1

In [ ]:
image_path = "/work/data/Lenna.png"
image.load_img(image_path)

In [ ]:
def calc_bandwidth(lambd, sigma):
    r = np.pi*sigma/lambd
    c = np.sqrt(np.log(2)/2)
    return np.log2((r + c)/(r - c))

def calc_sigma(lambd, bandwidth):
    p = 2**bandwidth
    c = np.sqrt(np.log(2)/2)
    return lambd * c / np.pi  * (p + 1) / (p - 1)

In [ ]:
for bw in np.linspace(0.4, 2.6, num=3):
    for lambd in range(3, 9):
        print(f"{bw:<3}, {lambd}, {calc_sigma(lambd, bw):<05.3}")

In [ ]:
for sigma in [1.5, 2, 2.5, 3]:
    for lambd in range(3, 9):
        print(f"{sigma:<3}, {lambd}, {calc_bandwidth(lambd, sigma):<05.3}")

In [ ]:
for sigma in [2, 3, 4, 5]:
#     for bw in np.linspace(0.4, 2.6, num=3):
    for bw in np.linspace(1, 1.8, num=5):
        print(f"{sigma:<3}, {bw}, {calc_lambda(sigma, bw):<05.3}")